In [10]:
import numpy as np
import theano.tensor as T
from theano import function

In [11]:
x = T.dscalar('x')
y = T.dscalar('y')
z = x + y
f = function([x, y], z)

In [12]:
f(2, 3)

array(5.0)

In [13]:
np.allclose(f(16.3, 12.1), 28.4)

True

In [14]:
type(x)

theano.tensor.var.TensorVariable

In [15]:
x.type

TensorType(float64, scalar)

In [16]:
T.dscalar

TensorType(float64, scalar)

In [17]:
from theano import pp
print(pp(z))

(x + y)


In [18]:
np.allclose(z.eval({x : 16.3, y : 12.1}), 28.4)

True

In [21]:
x = T.dmatrix('x')
y = T.dmatrix('y')
z = x + y
f = function([x, y], z)
f([[1, 2],
   [3, 4]],
  
  [[10, 20],
   [30, 40]])

array([[ 11.,  22.],
       [ 33.,  44.]])

In [22]:
import theano

In [23]:
a = T.vector()
out = a + a ** 10
f = theano.function([a], out)
print(f([0, 1, 2]))

[    0.     2.  1026.]


In [24]:
x = T.dmatrix('x')
s = 1 / (1 + T.exp(-x))
logistic = theano.function([x], s)
logistic([[0, 1], [-1, -2]])

array([[ 0.5       ,  0.73105858],
       [ 0.26894142,  0.11920292]])

In [25]:
s2 = (1 + T.tanh(x / 2)) / 2
logistic2 = theano.function([x], s2)
logistic2([[0, -1], [-1, -2]])

array([[ 0.5       ,  0.26894142],
       [ 0.26894142,  0.11920292]])

In [27]:
a, b = T.dmatrices('a', 'b')
diff = a - b
abs_diff = abs(diff)
diff_squared = diff ** 2
f = theano.function([a, b], [diff, abs_diff, diff_squared])
f([[1, 1], [1, 1]], [[0, 1], [2, 3]])

[array([[ 1.,  0.],
        [-1., -2.]]), array([[ 1.,  0.],
        [ 1.,  2.]]), array([[ 1.,  0.],
        [ 1.,  4.]])]

In [28]:
x, y = T.dscalars('x', 'y')
z = x + y
f = function([x, theano.In(y, value=1)], z)
f(33)

array(34.0)

In [29]:
f(33, 2)

array(35.0)

In [34]:
x, y, w = T.dscalars('x', 'y', 'w')
z = (x + y) * w
f = function([x, theano.In(y, value=1), theano.In(w, value=2, name='w_by_name')], z)
f(33)

array(68.0)

In [35]:
f(33, w_by_name=1)

array(34.0)

In [36]:
f(33, w_by_name=1, y=0)

array(33.0)

In [37]:
from theano import shared
state = shared(0)
inc = T.iscalar('inc')
accumulator = function([inc], state, updates=[(state, state + inc)])

In [38]:
print(state.get_value())

0


In [39]:
accumulator(1)

array(0)

In [40]:
print(state.get_value())

1


In [41]:
accumulator(300)

array(1)

In [42]:
print(state.get_value())

301


In [43]:
state.set_value(233)

In [44]:
print(state.get_value())

233


In [46]:
fn_of_state = state * 2 + inc
foo = T.scalar(dtype=state.dtype)
skip_shared = function([inc, foo], fn_of_state, givens=[(state, foo)])
skip_shared(1, 3)

array(7)

In [50]:
print(state.get_value())

233


In [52]:
inc = T.iscalar('inc')
accumulator = function([inc], state, updates=[(state, state + inc)])
accumulator(10)

array(233)

In [53]:
print(state.get_value())

243


In [55]:
new_state = theano.shared(0)
new_accumulator = accumulator.copy(swap={state : new_state})
new_accumulator(100)

[array(0)]

In [56]:
print(new_state.get_value())

100


In [57]:
null_accumulator = accumulator.copy(delete_updates=True)
null_accumulator(9000)

[array(243)]

In [58]:
print(state.get_value())

243


In [59]:
from theano.tensor.shared_randomstreams import RandomStreams
srng = RandomStreams(seed=234)
rv_u = srng.uniform((2, 2))
rv_n = srng.normal((2, 2))
f = function([], rv_u)
g = function([], rv_n, no_default_updates=True)
nearly_zeros = function([], rv_u + rv_u - 2 * rv_u)

In [66]:
print(f())
print(f())
print(g())
print(g())

[[ 0.86342685  0.81031029]
 [ 0.86695784  0.6813093 ]]
[[ 0.70574274  0.80222456]
 [ 0.25976164  0.18285402]]
[[ 0.37328447 -0.65746672]
 [-0.36302373 -0.97484625]]
[[ 0.37328447 -0.65746672]
 [-0.36302373 -0.97484625]]


In [67]:
rng_val = rv_u.rng.get_value(borrow=True)
rng_val.seed(89234)
rv_u.rng.set_value(rng_val, borrow=True)

array([[ 0.31971415,  0.47584377],
       [ 0.24129163,  0.42046081]])

array([[ 0.37328447, -0.65746672],
       [-0.36302373, -0.97484625]])

array([[ 0.37328447, -0.65746672],
       [-0.36302373, -0.97484625]])

array([[ 0.,  0.],
       [ 0.,  0.]])